### Word Group In Youtube

In [498]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [499]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [500]:
# language pair
lang_folder = "German"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index
unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [501]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [502]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [503]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [504]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [505]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [506]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [507]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
...,...,...
274527,galapremiere,5
274528,gemurkse,5
274529,granet,5
274530,bürgernahen,5


In [508]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [509]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
5,nicht,3820673
6,die,3473088
7,und,3173268
8,es,3167627
9,der,2415780


In [510]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [511]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [512]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

28

#### Youtube Sentence Analysis

In [513]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:14.080,00:00:22.080,in mitten der hamburger speicherstadt dem unes...,yN0AjDfDKBw
1,00:00:22.500,00:00:25.680,in,yN0AjDfDKBw
2,00:00:26.420,00:00:34.960,ist auf,yN0AjDfDKBw
3,00:00:35.740,00:00:38.440,die beliebteste touristenattraktion deutschlands,yN0AjDfDKBw
4,00:00:38.440,00:00:42.820,hält den guiness weltrekord als größte modelle...,yN0AjDfDKBw
...,...,...,...,...
1385250,00:08:39.050,00:08:46.080,bei wwwscooterexpertende und falls ihr noch me...,gpEE6RI3Io0
1385251,00:08:46.080,00:08:47.080,unseren kanal,gpEE6RI3Io0
1385252,00:08:47.080,00:08:52.370,und natürlich das letzte falls ihr wissen woll...,gpEE6RI3Io0
1385253,00:08:52.370,00:08:58.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0


In [514]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [515]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,14.08,22.080,in mitten der hamburger speicherstadt dem unes...,yN0AjDfDKBw
1,1,22.50,25.680,in,yN0AjDfDKBw
2,2,26.42,34.960,ist auf,yN0AjDfDKBw
3,3,35.74,38.440,die beliebteste touristenattraktion deutschlands,yN0AjDfDKBw
4,4,38.44,42.820,hält den guiness weltrekord als größte modelle...,yN0AjDfDKBw
...,...,...,...,...,...
1385250,1385250,519.05,526.080,bei wwwscooterexpertende und falls ihr noch me...,gpEE6RI3Io0
1385251,1385251,526.08,527.080,unseren kanal,gpEE6RI3Io0
1385252,1385252,527.08,532.370,und natürlich das letzte falls ihr wissen woll...,gpEE6RI3Io0
1385253,1385253,532.37,538.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0


In [516]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [517]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [518]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,14.08,22.080,in mitten der hamburger speicherstadt dem unes...,yN0AjDfDKBw,20.0,"hamburger, unesco, weltkulturerbe, mitten, spe...","der, in"
1,1,22.50,25.680,in,yN0AjDfDKBw,99.9,,in
2,2,26.42,34.960,ist auf,yN0AjDfDKBw,100.0,,"auf, ist"
3,3,35.74,38.440,die beliebteste touristenattraktion deutschlands,yN0AjDfDKBw,25.0,"deutschlands, beliebteste, touristenattraktion",die
4,4,38.44,42.820,hält den guiness weltrekord als größte modelle...,yN0AjDfDKBw,22.2,"welt, weltrekord, als, hält, modelleisenbahn, ...","der, den"
...,...,...,...,...,...,...,...,...
1385250,1385250,519.05,526.080,bei wwwscooterexpertende und falls ihr noch me...,gpEE6RI3Io0,7.7,"uns, wwwscooterexpertende, von, falls, abonnie...",und
1385251,1385251,526.08,527.080,unseren kanal,gpEE6RI3Io0,0.0,"unseren, kanal",
1385252,1385252,527.08,532.370,und natürlich das letzte falls ihr wissen woll...,gpEE6RI3Io0,33.3,"man, letzte, ihr, falls, versicherung, escoote...","zu, das, und, eine, wie"
1385253,1385253,532.37,538.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0,15.4,"video, dafür, da, drauf, haben, klickt, findet...","ein, wir"


In [519]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,14.08,22.080,in mitten der hamburger speicherstadt dem unes...,yN0AjDfDKBw,20.0,"hamburger, unesco, weltkulturerbe, mitten, spe...","der, in"
1,1,22.50,25.680,in,yN0AjDfDKBw,99.9,NaN,in
2,2,26.42,34.960,ist auf,yN0AjDfDKBw,100.0,NaN,"auf, ist"
3,3,35.74,38.440,die beliebteste touristenattraktion deutschlands,yN0AjDfDKBw,25.0,"deutschlands, beliebteste, touristenattraktion",die
4,4,38.44,42.820,hält den guiness weltrekord als größte modelle...,yN0AjDfDKBw,22.2,"welt, weltrekord, als, hält, modelleisenbahn, ...","der, den"
...,...,...,...,...,...,...,...,...
1385250,1385250,519.05,526.080,bei wwwscooterexpertende und falls ihr noch me...,gpEE6RI3Io0,7.7,"uns, wwwscooterexpertende, von, falls, abonnie...",und
1385251,1385251,526.08,527.080,unseren kanal,gpEE6RI3Io0,0.0,"unseren, kanal",NaN
1385252,1385252,527.08,532.370,und natürlich das letzte falls ihr wissen woll...,gpEE6RI3Io0,33.3,"man, letzte, ihr, falls, versicherung, escoote...","zu, das, und, eine, wie"
1385253,1385253,532.37,538.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0,15.4,"video, dafür, da, drauf, haben, klickt, findet...","ein, wir"


In [520]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,14.08,22.080,in mitten der hamburger speicherstadt dem unes...,yN0AjDfDKBw,20.0,"hamburger, unesco, weltkulturerbe, mitten, spe...","der, in"
1,1,22.50,25.680,in,yN0AjDfDKBw,99.9,NaN,in
2,2,26.42,34.960,ist auf,yN0AjDfDKBw,100.0,NaN,"auf, ist"
3,3,35.74,38.440,die beliebteste touristenattraktion deutschlands,yN0AjDfDKBw,25.0,"deutschlands, beliebteste, touristenattraktion",die
4,4,38.44,42.820,hält den guiness weltrekord als größte modelle...,yN0AjDfDKBw,22.2,"welt, weltrekord, als, hält, modelleisenbahn, ...","der, den"
...,...,...,...,...,...,...,...,...
1032317,1385247,503.55,508.730,ich hoffe es hat euch gefallen und falls es eu...,gpEE6RI3Io0,29.4,"like, uns, falls, gerne, gefallen, euch, hoffe...","ein, hat, ich, es, und"
1032318,1385248,508.73,513.910,falls ihr ideen anmerkungen oder news habt sch...,gpEE6RI3Io0,20.0,"drunter, uns, habt, da, oder, falls, kommentar...","in, es, das"
1032319,1385252,527.08,532.370,und natürlich das letzte falls ihr wissen woll...,gpEE6RI3Io0,33.3,"man, letzte, ihr, falls, versicherung, escoote...","zu, das, und, eine, wie"
1032320,1385253,532.37,538.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0,15.4,"video, dafür, da, drauf, haben, klickt, findet...","ein, wir"


In [521]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-521-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
2,2,26.42,34.960,ist auf,yN0AjDfDKBw,100.0,NaN,"auf, ist",ist auf
5,6,49.56,57.740,ein überwältigendes erlebnis für alle verspiel...,yN0AjDfDKBw,31.2,"überwältigendes, riesige, erlebnis, abenteurer...","mit, ein, die, eine, in",die in eine
8,10,79.58,87.500,auf der nordostsee schwimmen schiffe auf echtw...,yN0AjDfDKBw,33.3,"echtwasser, einem, schiffe, schwimmen, nordostsee","der, auf, in",auf der
11,14,119.50,128.620,knuffingen airport ist laut guiness world reco...,yN0AjDfDKBw,20.0,"world, bis, laut, technisch, welt, letzte, knu...","der, ist, in, und",und ist
12,20,162.76,166.640,sehr lebendige menschen und nicht mehr so lebe...,yN0AjDfDKBw,37.5,"lebendige, mehr, sehr, menschen","nicht, so, und",und nicht
...,...,...,...,...,...,...,...,...,...
1032316,1385246,501.00,503.550,so das waren die scooter news dieser woche,gpEE6RI3Io0,37.5,"scooter, dieser, woche, news, waren","die, so, das",so das
1032317,1385247,503.55,508.730,ich hoffe es hat euch gefallen und falls es eu...,gpEE6RI3Io0,29.4,"like, uns, falls, gerne, gefallen, euch, hoffe...","ein, hat, ich, es, und",es hat
1032318,1385248,508.73,513.910,falls ihr ideen anmerkungen oder news habt sch...,gpEE6RI3Io0,20.0,"drunter, uns, habt, da, oder, falls, kommentar...","in, es, das",in das
1032320,1385253,532.37,538.130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0,15.4,"video, dafür, da, drauf, haben, klickt, findet...","ein, wir",wir ein


In [522]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ist auf,26.420000,34.960000,ist auf,yN0AjDfDKBw
1,die in eine,53.582951,54.454590,ein überwältigendes erlebnis für alle verspiel...,yN0AjDfDKBw
2,auf der,79.580000,80.636000,auf der nordostsee schwimmen schiffe auf echtw...,yN0AjDfDKBw
3,und ist,125.183478,125.778261,knuffingen airport ist laut guiness world reco...,yN0AjDfDKBw
4,und nicht,164.509804,165.346667,sehr lebendige menschen und nicht mehr so lebe...,yN0AjDfDKBw
...,...,...,...,...,...
520105,so das,501.000000,501.425000,so das waren die scooter news dieser woche,gpEE6RI3Io0
520106,es hat,504.098471,504.586000,ich hoffe es hat euch gefallen und falls es eu...,gpEE6RI3Io0
520107,in das,512.068222,512.528667,falls ihr ideen anmerkungen oder news habt sch...,gpEE6RI3Io0
520108,wir ein,534.957826,535.709130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0


In [523]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ist auf,26.120000,35.260000,ist auf,yN0AjDfDKBw
1,die in eine,53.282951,54.754590,ein überwältigendes erlebnis für alle verspiel...,yN0AjDfDKBw
2,auf der,79.280000,80.936000,auf der nordostsee schwimmen schiffe auf echtw...,yN0AjDfDKBw
3,und ist,124.883478,126.078261,knuffingen airport ist laut guiness world reco...,yN0AjDfDKBw
4,und nicht,164.209804,165.646667,sehr lebendige menschen und nicht mehr so lebe...,yN0AjDfDKBw
...,...,...,...,...,...
520105,so das,500.700000,501.725000,so das waren die scooter news dieser woche,gpEE6RI3Io0
520106,es hat,503.798471,504.886000,ich hoffe es hat euch gefallen und falls es eu...,gpEE6RI3Io0
520107,in das,511.768222,512.828667,falls ihr ideen anmerkungen oder news habt sch...,gpEE6RI3Io0
520108,wir ein,534.657826,536.009130,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0


In [524]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ist auf,26,35,ist auf,yN0AjDfDKBw
1,die in eine,53,55,ein überwältigendes erlebnis für alle verspiel...,yN0AjDfDKBw
2,auf der,79,81,auf der nordostsee schwimmen schiffe auf echtw...,yN0AjDfDKBw
3,und ist,125,126,knuffingen airport ist laut guiness world reco...,yN0AjDfDKBw
4,und nicht,164,166,sehr lebendige menschen und nicht mehr so lebe...,yN0AjDfDKBw
...,...,...,...,...,...
520105,so das,501,502,so das waren die scooter news dieser woche,gpEE6RI3Io0
520106,es hat,504,505,ich hoffe es hat euch gefallen und falls es eu...,gpEE6RI3Io0
520107,in das,512,513,falls ihr ideen anmerkungen oder news habt sch...,gpEE6RI3Io0
520108,wir ein,535,536,findet klickt da drauf da haben wir ein video ...,gpEE6RI3Io0


In [525]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,nicht aber ist das ist das ist das ist ja mit,901,905,schlagzeuger ein jahr früher also jetzt nicht ...,J2xdS5NrneA
1,ist ja nicht so dass ich nicht auf eine,795,798,kann mich anpassen also ist ja nicht so dass i...,wljYu2x6gBk
2,ist dass das ist das ist das ist die zu,1235,1238,sich auch lohnen ist dass die karaffe oder ist...,6sMVM6Ll0ps
3,ist es so und so und so und so und der,76,79,ich habe ganz oft schon immer gehört beim ster...,LYXkDy9P8JQ
4,ist ja ist es ist das ist das ist das,222,226,ist ja ist es ist das ist das ist das nichts s...,wYNv75Z5E0g
...,...,...,...,...,...
520105,es in,395,396,nun stabiler als je zuvor wir haben es in,psrLJIswCgg
520106,ja es,581,582,ja es gibt noch viele offene fragen,HhQPcrFzWKA
520107,du in,68,68,frag dich auch welche position du in deinem un...,UlMkxM7Bmug
520108,es zu,38,39,ohne es zu wissen oder sogar ich,Mk9sfxLsXr0


In [526]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,nicht aber ist das ist das ist das ist ja mit,901,905,schlagzeuger ein jahr früher also jetzt nicht ...,J2xdS5NrneA,54
1,ist ja nicht so dass ich nicht auf eine,795,798,kann mich anpassen also ist ja nicht so dass i...,wljYu2x6gBk,88
2,ist dass das ist das ist das ist die zu,1235,1238,sich auch lohnen ist dass die karaffe oder ist...,6sMVM6Ll0ps,50
3,ist es so und so und so und so und der,76,79,ich habe ganz oft schon immer gehört beim ster...,LYXkDy9P8JQ,45
4,ist ja ist es ist das ist das ist das,222,226,ist ja ist es ist das ist das ist das nichts s...,wYNv75Z5E0g,40
...,...,...,...,...,...,...
520105,es in,395,396,nun stabiler als je zuvor wir haben es in,psrLJIswCgg,100
520106,ja es,581,582,ja es gibt noch viele offene fragen,HhQPcrFzWKA,100
520107,du in,68,68,frag dich auch welche position du in deinem un...,UlMkxM7Bmug,100
520108,es zu,38,39,ohne es zu wissen oder sogar ich,Mk9sfxLsXr0,100


In [527]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-527-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-527-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,ist ja nicht so dass ich nicht auf eine,795,798,kann mich anpassen also ist ja nicht so dass i...,wljYu2x6gBk
1,das ist nicht so dass das so wie eine,1051,1054,also das ist nicht so dass das so wie eine cnc...,U2W6SoY5rts
2,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4
3,aber es ist nicht so dass er so was,1326,1328,aber es ist nicht so dass er so was rein balle...,HrPhl3GqNxw
4,nicht aber es ist nicht so dass sie,103,106,also sie mögen auch sich selbst nicht aber es ...,hVJhdckdKYs
...,...,...,...,...,...
363642,so in,133,134,er unterstützung an dem tag braucht und dann s...,aH_0vau4sMQ
363643,du in,376,378,at nutzen falls du in irgendeinem zeitpunkt ni...,psrLJIswCgg
363644,es in,395,396,nun stabiler als je zuvor wir haben es in,psrLJIswCgg
363645,ja es,581,582,ja es gibt noch viele offene fragen,HhQPcrFzWKA


In [528]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-528-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,ist ja nicht so dass ich nicht auf eine,795,798,kann mich anpassen also ist ja nicht so dass i...,wljYu2x6gBk,40
1,das ist nicht so dass das so wie eine,1051,1054,also das ist nicht so dass das so wie eine cnc...,U2W6SoY5rts,58
2,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,80
3,aber es ist nicht so dass er so was,1326,1328,aber es ist nicht so dass er so was rein balle...,HrPhl3GqNxw,47
4,nicht aber es ist nicht so dass sie,103,106,also sie mögen auch sich selbst nicht aber es ...,hVJhdckdKYs,48
...,...,...,...,...,...,...
363642,so in,133,134,er unterstützung an dem tag braucht und dann s...,aH_0vau4sMQ,6
363643,du in,376,378,at nutzen falls du in irgendeinem zeitpunkt ni...,psrLJIswCgg,5
363644,es in,395,396,nun stabiler als je zuvor wir haben es in,psrLJIswCgg,12
363645,ja es,581,582,ja es gibt noch viele offene fragen,HhQPcrFzWKA,14


In [529]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-529-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4
1,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k
2,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY
3,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs
4,aber ist es nicht so dass du,353,354,aber ist es nicht so dass du eigentlich,X10OIhdciIw
...,...,...,...,...,...
2076,so ja,781,783,so ja,d0wfLjKdH6c
2077,ja so,1857,1860,ja so,AL5TzaInG8Y
2078,so in,170,174,so in,AmeiJYUJ7s4
2079,ja in,472,475,ja in,hemhSkcVLcM


In [530]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,ich aber nicht,zom1kfDBHv8
1,aber die,zikVfJgpYT4
2,in eine der,zeFYZvdJXaI
3,das nicht das ist was wir,zReDxW01GDc
4,die hat,zP7LKGqz97w
...,...,...
556,ja und die,0vUAGvp9Gd4
557,die du mir,0tn9vp3dLUk
558,du eine,0WWayLM3Gt8
559,und was ist es nicht,0TpAlYbiMIs


In [531]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4
1,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k
2,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY
3,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs
4,aber ist es nicht so dass du,353,354,aber ist es nicht so dass du eigentlich,X10OIhdciIw
...,...,...,...,...,...
545,in der,183,190,in der,iNEXCSKFVCY
546,in die,1564,1569,in die,NBn0ZUZLvus
547,was ja,368,371,was ja,utgP1otuPEQ
548,in den,504,507,in den,uNrn4GI1q1E


In [532]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,ist sie aber so aber was ist mit mir,644,648,und dann ist sie aber so aber was ist mit mir,E5wrYo4PLM4,https://www.youtube.com/watch?v=E5wrYo4PLM4&t=...
1,nicht dass das so was ist das,687,690,oh nee sag nicht dass das so was ist das,m9hZzWA0R3k,https://www.youtube.com/watch?v=m9hZzWA0R3k&t=...
2,das ist ein dass das nicht so,578,580,das ist ein dass das nicht so,XGhAlBQ3gOY,https://www.youtube.com/watch?v=XGhAlBQ3gOY&t=...
3,das ist ja nicht das was ich,548,551,das ist ja nicht das was ich,X0g4THI7vEs,https://www.youtube.com/watch?v=X0g4THI7vEs&t=...
4,aber ist es nicht so dass du,353,354,aber ist es nicht so dass du eigentlich,X10OIhdciIw,https://www.youtube.com/watch?v=X10OIhdciIw&t=...
...,...,...,...,...,...,...
545,in der,183,190,in der,iNEXCSKFVCY,https://www.youtube.com/watch?v=iNEXCSKFVCY&t=...
546,in die,1564,1569,in die,NBn0ZUZLvus,https://www.youtube.com/watch?v=NBn0ZUZLvus&t=...
547,was ja,368,371,was ja,utgP1otuPEQ,https://www.youtube.com/watch?v=utgP1otuPEQ&t=...
548,in den,504,507,in den,uNrn4GI1q1E,https://www.youtube.com/watch?v=uNrn4GI1q1E&t=...


In [533]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [534]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [535]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [536]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['German_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'German_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'German_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx']

In [537]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [538]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass